In [ ]:
def preprocess_folder(raw_path, result_path):
    labels = os.listdir(raw_path)
    
    raw_counts = dict()
    for i in labels:
        if i != '.DS_Store':
            raw_counts[i] = len(os.listdir(f"{raw_path}/{i}"))
    
    labels_count = dict()
    for name in tqdm(glob.glob(raw_path+'/*'+'/*.png')):
        os.chdir(result_path)
        print(name)
        break
        
        
        label = name.rpartition('_')[0]
        _, label = os.path.split(label)
        if label not in labels_count.keys():
            labels_count[label] = 1
            os.mkdir(label)
        else:
            labels_count[label] += 1
        filename = f'{label}_{labels_count[label]}'

        img = Image.open(name)
        if img.size == (41, 40):
            continue
        img_final = np.array(pure_pil_alpha_to_color(img))
        # if np.random.choice([0,1]) == 0:
        #     img_final = crop_borders(apply_gauss(np.array(ImageOps.invert(img))))
        # else:
        # img_final = apply_gauss(np.array(img))
        
        
        # os.chdir(f'{result_path}/{label}')
        # if len(os.listdir(f'{raw_path}/{label}')) <= 8:
        #     augment(cv2.cvtColor(img_final, cv2.COLOR_BGR2RGB), filename)
        # else:
        #     plt.imsave(f"{filename}_{str(labels_count[label])}.jpg", img_final)
            
            
        count_to_add = 50 - raw_counts[label]
        os.chdir(f'{result_path}/{label}')
        augment(cv2.cvtColor(img_final, cv2.COLOR_BGR2RGB), filename, size=count_to_add//4)
            
            
        remove_empty(result_path)
        
    return labels_count

In [ ]:
import glob
import os
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from PIL import Image, ImageOps
import albumentations as A
from albumentations import Blur

In [ ]:
def pure_pil_alpha_to_color(image, color=(255, 255, 255)):
    """Alpha composite an RGBA Image with a specified color.
    Source: http://stackoverflow.com/a/9459208/284318
    """
    image.load()  # needed for split()
    background = Image.new('RGB', image.size, color)
    background.paste(image, mask=image.split()[3])  # 3 is the alpha channel
    return background

In [ ]:
def apply_gauss(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    _, img = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return img

In [ ]:
def extract_all_squares(image, kernel_length):
    """
    Binarizes image, keeping only vertical and horizontal lines
    hopefully, it'll help us detect squares
    Args:
        image: image (cropped around circonstances)
        kernel_length: length of kernel to use. Too long and you will catch everything,
            too short and you catch nothing
    Returns:
        image binarized and keeping only vertical and horizozntal lines
    """
    # thresholds image : anything beneath a certain value is set to zero
    (thresh, img_bin) = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    # A vertical kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
    vertical_ksize = (1, kernel_length)
    # Morphological operation to detect vertical lines from an image
    verticle_lines_img = extract_lines(img_bin, vertical_ksize)

    # A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
    horizontal_ksize = (kernel_length, 1)
    # Morphological operation to detect horizontal lines from an image
    horizontal_lines_img = extract_lines(img_bin, horizontal_ksize)
    img_final_bin = add_lines_together(verticle_lines_img, horizontal_lines_img)

    return img_final_bin


def extract_lines(image, ksize):
    """
    extract lines (horizontal or vertical, depending on ksize)
    Args:
        image: binarized image
        ksize: size of kernel to use. Possible values :
            horizontal_ksize = (kernel_length, 1)
            vertical_ksize = (1, kernel_length)
    Returns:
        lines from image (vertical or horizontal, depending on ksize)
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, ksize)
    img_temp = cv2.erode(image, kernel, iterations=3)
    lines_img = cv2.dilate(img_temp, kernel, iterations=3)
    return lines_img


def add_lines_together(verticle_lines_img, horizontal_lines_img, alpha=0.5, beta=0.5):
    """
    extract lines (horizontal or vertical, depending on ksize)
    Args:
        verticle_lines_img: image with vertical lines
        horizontal_lines_img: image with horizontal lines
        alpha : weight of first image. Keep at 0.5 for balance
        beta : weight of second image. Keep at 0.5 for balance
            alpha and beta are weighting parameters, this will
            decide the quantity of an image to be added to make a new image
    Returns:
        image with an addition of both vertical and horizontal lines
    """

    # This function helps to add two image with specific weight parameter to get a third image as summation of two image.
    img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
    # A kernel of (3 X 3) nes.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    # erodes boundaries of features, gets rid of some noise
    img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
    # further kill noise by thresholding
    (thresh, img_final_bin) = cv2.threshold(img_final_bin, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return img_final_bin

In [ ]:
def crop_borders(image):
    # extract horizontal and vertical lines
    only_box = extract_all_squares(image, kernel_length=50)
    # build up a mask of the same size as the image
    mask = np.zeros(image.shape, dtype='uint8')
    # get contours of horizontal and vetical lines
    contours, hierarchy = cv2.findContours(only_box, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # draw contours on mask
    mask = cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)
    # threhold mask and image
    ret, mask = cv2.threshold(mask, 20, 255, cv2.THRESH_BINARY)
    ret, box = cv2.threshold(image, 20, 255, cv2.THRESH_BINARY)
    # remove the bits we don't want
    box[mask == 0] = 255
    return box


In [ ]:
def is_too_black(image, thresh):
    """
    Args:
        image - PIL image
        thresh - threshold
    Returns:
        image if threshold is not exceeded
        else None
    """
    
    pixels = image.getdata()
    nblack = 0
    for pixel in pixels:
        if pixel[0] < thresh:
            nblack += 1
    n = len(pixels)

    if (nblack / float(n)) > 0.37:
        return True
    return False

In [9]:
# adds fog, blur, noise, changes brightness and contrast (all at random)
transformer_fogger = A.Compose([
    A.augmentations.transforms.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.5, alpha_coef=0.7, p=1),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.6),
    Blur(blur_limit=3, p=0.3),
    A.augmentations.transforms.GaussNoise(var_limit=(10.0, 50.0), mean=0, per_channel=True, p=0.4)
])

# adds snow, blur, noise, changes brightness and contrast (all at random)
transformer_snower = A.Compose([
    A.augmentations.transforms.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.6, brightness_coeff=1.5, p=1),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.6),
    Blur(blur_limit=3, p=0.3),
    A.augmentations.transforms.GaussNoise(var_limit=(10.0, 50.0), mean=0, per_channel=True, p=0.4)
])

# adds blur, noise, changes brightness, rotates and scales image (all at random)
transformer_rotater = A.Compose([
    A.augmentations.geometric.rotate.Rotate(limit=[-30, 30], p=1),
    A.augmentations.geometric.resize.RandomScale(0.3, p=1),
    Blur(blur_limit=3, p=0.3),
    A.augmentations.transforms.GaussNoise(var_limit=(10.0, 50.0), mean=0, per_channel=True, p=0.4),
    A.augmentations.transforms.RandomBrightness(limit=0.4, p=0.5),
    A.augmentations.transforms.InvertImg()
])

# adds blur, noise, changes brightness and applies geometric transform (all at random)
transformer_affine = A.Compose([
    A.augmentations.geometric.transforms.Affine(p=1),
    Blur(blur_limit=3, p=0.3),
    A.augmentations.transforms.GaussNoise(var_limit=(10.0, 50.0), mean=0, per_channel=True, p=0.4),
    A.augmentations.transforms.RandomBrightness(limit=0.4, p=0.7),
    A.augmentations.transforms.InvertImg()
])

# adds blur, noise, changes brightness and perspective (all at random)
transformer_perspective = A.Compose([
    A.augmentations.geometric.transforms.Perspective(p=1),
    Blur(blur_limit=3, p=0.3),
    A.augmentations.transforms.GaussNoise(var_limit=(10.0, 50.0), mean=0, per_channel=True, p=0.4),
    A.augmentations.transforms.RandomBrightness(limit=0.4, p=0.7),
    A.augmentations.transforms.InvertImg()
])


trans_map = {1: transformer_fogger,
           2: transformer_perspective,
           3: transformer_rotater,
           4: transformer_affine}

def create_augment_block(transformer, img):
    thresh = transformer(image=img)
    image_transformed = thresh["image"]
    return image_transformed

def augment(img, filename, size, verbose=False):
    transformers = np.random.randint(1, 5, size)
    for num, trans in enumerate(transformers):
        plt.imsave(f"{filename}_{str(num)}.jpg", create_augment_block(trans_map[trans], img))


In [10]:
def remove_empty(path):
    folders = glob.glob(os.path.join(path, '*'))
    for name in folders:
        if len(os.listdir(name)) == 0:
            os.rmdir(name)

# 50/50 invert, augment small classes

In [196]:
raw_path = '/Users/dushamishkin/Desktop/Jupyter/JinWen/Backup'
labels = os.listdir(raw_path)
for label in tqdm(labels):
    if label != '.DS_Store':
        for file in os.listdir(f"{raw_path}/{label}"):
            orig_filename = f'{raw_path}/{label}/{file}'
            img = Image.open(orig_filename)
            if img.size == (41, 40):
                os.remove(orig_filename)

  0%|          | 0/3263 [00:00<?, ?it/s]

In [11]:
def preprocess_folder(raw_path, result_path):
    remove_empty(raw_path)
    remove_empty(result_path)
    
    labels = os.listdir(raw_path)
    
    raw_counts = dict()
    for i in labels:
        if i != '.DS_Store':
            raw_counts[i] = len(os.listdir(f"{raw_path}/{i}"))
    
    labels_count = dict()
    for label in tqdm(labels):
        os.mkdir(f'{result_path}/{label}')
        if label != '.DS_Store':
            os.chdir(result_path)
            files = os.listdir(f"{raw_path}/{label}")
            
         
            num_to_add = (100 - raw_counts[label]) // raw_counts[label]
            # num_to_add = ceil(num_to_add // raw_counts[label] / 4)
        
            for file in os.listdir(f"{raw_path}/{label}"):  
                orig_filename = f'{raw_path}/{label}/{file}'
                img = Image.open(orig_filename)
                if img.size == (41, 40):
                    continue
                img_final = np.array(pure_pil_alpha_to_color(img))
                
                target_filename = f'{result_path}/{label}/{file}'[:-4]
                
                os.chdir(f'{result_path}/{label}')
                augment(cv2.cvtColor(img_final, cv2.COLOR_BGR2RGB), target_filename, size=num_to_add)

            
        
    return labels_count